Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Responsible AI dashboard for Time Series Forecasting
_**Orange Juice Sales Forecasting**_

## Contents
1. [Introduction](#introduction)
1. [Data](#data)
1. [Train](#train)
1. [Forecast](#forecast)
1. [Responsible AI Dashboard](#analyze)

## Introduction<a id="introduction"></a>
In this example, we use sktime to train, select, and operationalize a time-series forecasting model for multiple time-series.

The examples in the follow code samples use the University of Chicago's Dominick's Finer Foods dataset to forecast orange juice sales. Dominick's was a grocery chain in the Chicago metropolitan area.

In [16]:
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import PowerTransformer, RobustScaler, MinMaxScaler
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.transformations.series.adapt import TabularToSeriesAdaptor
from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.base import ForecastingHorizon

## Data<a id="data"></a>
You are now ready to load the historical orange juice sales data. We will load the CSV file into a plain pandas DataFrame; the time column in the CSV is called _WeekStarting_, so it will be specially parsed into the datetime type.

In [24]:
time_column_name = "WeekStarting"
time_series_id_column_names = ["Store", "Brand"]
dataset_location = "https://raw.githubusercontent.com/Azure/azureml-examples/2fe81643865e1f4591e7734bd1a729093cafb826/v1/python-sdk/tutorials/automl-with-azureml/forecasting-orange-juice-sales/dominicks_OJ.csv"
data = pd.read_csv(dataset_location, parse_dates=[time_column_name])

# Drop the columns 'logQuantity' as it is a leaky feature.
data.drop("logQuantity", axis=1, inplace=True)

data.set_index(time_series_id_column_names + [time_column_name], inplace=True, ascending=[True, True, True])

data.head()

Quantity  Advert  Price     Age60   COLLEGE  \
Store Brand       WeekStarting                                                
2     dominicks   1990-06-14       10560       1   1.59  0.232865  0.248935   
      minute.maid 1990-06-14        4480       0   3.17  0.232865  0.248935   
      tropicana   1990-06-14        8256       0   3.87  0.232865  0.248935   
5     dominicks   1990-06-14        1792       1   1.59  0.117368  0.321226   
      minute.maid 1990-06-14        4224       0   2.99  0.117368  0.321226   

                                   INCOME  Hincome150  Large HH  Minorities  \
Store Brand       WeekStarting                                                
2     dominicks   1990-06-14    10.553205    0.463887  0.103953    0.114280   
      minute.maid 1990-06-14    10.553205    0.463887  0.103953    0.114280   
      tropicana   1990-06-14    10.553205    0.463887  0.103953    0.114280   
5     dominicks   1990-06-14    10.922371    0.535883  0.103092    0.053875   
      minute.maid 1990-06-14    10.922371    0.535883  0.103092    0.053875   

                                WorkingWoman  SSTRDIST   SSTRVOL   CPDIST5  \
Store Brand       WeekStarting                                               
2     dominicks   1990-06-14        0.303585  2.110122  1.142857  1.927280   
      minute.maid 1990-06-14        0.303585  2.110122  1.142857  1.927280   
      tropicana   1990-06-14        0.303585  2.110122  1.142857  1.927280   
5     dominicks   1990-06-14        0.410568  3.801998  0.681818  1.600573   
      minute.maid 1990-06-14        0.410568  3.801998  0.681818  1.600573   

                                 CPWVOL5  
Store Brand       WeekStarting            
2     dominicks   1990-06-14    0.376927  
      minute.maid 1990-06-14    0.376927  
      tropicana   1990-06-14    0.376927  
5     dominicks   1990-06-14    0.736307  
      minute.maid 1990-06-14    0.736307

Each row in the DataFrame holds a quantity of weekly sales for an OJ brand at a single store. The data also includes the sales price, a flag indicating if the OJ brand was advertised in the store that week, and some customer demographic information based on the store location. For historical reasons, the data also include the logarithm of the sales quantity. The Dominick's grocery data is commonly used to illustrate econometric modeling techniques where logarithms of quantities are generally preferred.    

The task is now to build a time-series model for the _Quantity_ column. It is important to note that this dataset is comprised of many individual time-series - one for each unique combination of _Store_ and _Brand_. To distinguish the individual time-series, we define the **time_series_id_column_names** - the columns whose values determine the boundaries between time-series: 

In [49]:
nseries = data.groupby(time_series_id_column_names).ngroups
print("Data contains {0} individual time-series.".format(nseries))
data.loc[(slice(2,5), "dominicks", "1990-06-14")]

Data contains 249 individual time-series.


UnsortedIndexError: 'MultiIndex slicing requires the index to be lexsorted: slicing on levels [0], lexsort depth 0'

For demonstration purposes, we extract sales time-series for just a few of the stores:

In [42]:
use_stores = [2, 5, 8]
data_subset = data.loc[2]
nseries = data_subset.groupby(time_series_id_column_names).ngroups
print("Data subset contains {0} individual time-series.".format(nseries))

KeyError: 'Store'

### Data Splitting
We now split the data into a training and a testing set for later forecast evaluation. The test set will contain the final 20 weeks of observed sales for each time-series. The splits should be stratified by series, so we use a group-by statement on the time series identifier columns.

In [5]:
n_test_periods = 20

def split_last_n_by_series_id(df, n):
    """Group df by series identifiers and split on last n rows for each group."""
    df_grouped = df.sort_values(time_column_name).groupby(  # Sort by ascending time
        time_series_id_column_names, group_keys=False
    )
    df_head = df_grouped.apply(lambda dfg: dfg.iloc[:-n])
    df_tail = df_grouped.apply(lambda dfg: dfg.iloc[-n:])
    return df_head, df_tail

train, test = split_last_n_by_series_id(data_subset, n_test_periods)

## Modeling

For forecasting tasks, we need to do several pre-processing and estimation steps that are specific to time-series including:
* Detect time-series sample frequency (e.g. hourly, daily, weekly) and create new records for absent time points to make the series regular. A regular time series has a well-defined frequency and has a value at every sample point in a contiguous time span 
* Impute missing values in the target (via forward-fill) and feature columns (using median column values) 
* Create features based on time series identifiers to enable fixed effects across different series
* Create time-based features to assist in learning seasonal patterns
* Encode categorical variables to numeric quantities

In this notebook, we will train a single, regression-type model across **all** time-series in a given training set. This allows the model to generalize across related series.

In [6]:
target_column_name = "Quantity"

## Customization

The featurization customization in forecasting is an advanced feature in AutoML which allows our customers to change the default forecasting featurization behaviors and column types through `FeaturizationConfig`. The supported scenarios include:

1. Column purposes update: Override feature type for the specified column. Currently supports DateTime, Categorical and Numeric. This customization can be used in the scenario that the type of the column cannot correctly reflect its purpose. Some numerical columns, for instance, can be treated as Categorical columns which need to be converted to categorical while some can be treated as epoch timestamp which need to be converted to datetime. To tell our SDK to correctly preprocess these columns, a configuration need to be add with the columns and their desired types.
2. Transformer parameters update: Currently supports parameter change for Imputer only. User can customize imputation methods. The supported imputing methods for target column are constant and ffill (forward fill). The supported imputing methods for feature columns are mean, median, most frequent, constant and ffill (forward fill). This customization can be used for the scenario that our customers know which imputation methods fit best to the input data. For instance, some datasets use NaN to represent 0 which the correct behavior should impute all the missing value with 0. To achieve this behavior, these columns need to be configured as constant imputation with `fill_value` 0.
3. Drop columns: Columns to drop from being featurized. These usually are the columns which are leaky or the columns contain no useful data.

In [15]:
forecaster = TransformedTargetForecaster(steps=[
    ("detrend", Detrender()),
    ("deseasonalize", Deseasonalizer()),
    ("minmax", TabularToSeriesAdaptor(MinMaxScaler((1, 10)))),
    ("power", TabularToSeriesAdaptor(PowerTransformer())),
    ("scaler", TabularToSeriesAdaptor(RobustScaler())),
    ("forecaster", AutoARIMA(sp=1, suppress_warnings=True)),
])

fh = ForecastingHorizon(pd.DatetimeIndex(np.unique(test[time_column_name])), is_relative=False)

"""
featurization_config = FeaturizationConfig()
# Force the CPWVOL5 feature to be numeric type.
featurization_config.add_column_purpose("CPWVOL5", "Numeric")
# Fill missing values in the target column, Quantity, with zeros.
featurization_config.add_transformer_params(
    "Imputer", ["Quantity"], {"strategy": "constant", "fill_value": 0}
)
# Fill missing values in the INCOME column with median value.
featurization_config.add_transformer_params(
    "Imputer", ["INCOME"], {"strategy": "median"}
)
# Fill missing values in the Price column with forward fill (last value carried forward).
featurization_config.add_transformer_params("Imputer", ["Price"], {"strategy": "ffill"})
"""
fh

ForecastingHorizon(['1992-05-14', '1992-05-21', '1992-05-28', '1992-06-04',
               '1992-06-11', '1992-06-18', '1992-06-25', '1992-07-02',
               '1992-07-09', '1992-07-16', '1992-07-23', '1992-07-30',
               '1992-08-06', '1992-08-13', '1992-08-20', '1992-08-27',
               '1992-09-03', '1992-09-10', '1992-09-17', '1992-09-24',
               '1992-10-01'],
              dtype='datetime64[ns]', freq=None, is_relative=False)

## Forecasting Parameters
To define forecasting parameters for your experiment training, you can leverage the ForecastingParameters class. The table below details the forecasting parameter we will be passing into our experiment.


|Property|Description|
|-|-|
|**time_column_name**|The name of your time column.|
|**forecast_horizon**|The forecast horizon is how many periods forward you would like to forecast. This integer horizon is in units of the timeseries frequency (e.g. daily, weekly).|
|**time_series_id_column_names**|The column names used to uniquely identify the time series in data that has multiple rows with the same timestamp. If the time series identifiers are not defined, the data set is assumed to be one time series.|
|**freq**|Forecast frequency. This optional parameter represents the period with which the forecast is desired, for example, daily, weekly, yearly, etc. Use this parameter for the correction of time series containing irregular data points or for padding of short time series. The frequency needs to be a pandas offset alias. Please refer to [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects) for more information.
|**cv_step_size**|Number of periods between two consecutive cross-validation folds. The default value is "auto", in which case AutoMl determines the cross-validation step size automatically, if a validation set is not provided. Or users could specify an integer value.

## Train<a id="train"></a>

You can now submit a new training run. Depending on the data and number of iterations this operation may take several minutes.
Information from each iteration will be printed to the console.  Validation errors and current status will be shown when setting `show_output=True` and the execution will be synchronous.

In [21]:
forecaster.fit(y=train[target_column_name], X=train.drop(columns=[target_column_name]), fh=fh)
forecaster.predict(fh, test.drop(target_column_name))

TypeError: y must be in an sktime compatible format, of scitype Series, Panel or Hierarchical, for instance a pandas.DataFrame with sktime compatible time indices, or with MultiIndex and last(-1) level an sktime compatible time index. See the forecasting tutorial examples/01_forecasting.ipynb, or the data format tutorial examples/AA_datatypes_and_datasets.ipynb,If you think y is already in an sktime supported input format, run sktime.datatypes.check_raise(y, mtype) to diagnose the error, where mtype is the string of the type specification you want for y. Possible mtype specification strings are as follows. "For Hierarchical scitype: ['pd_multiindex_hier']. 

# Forecast<a id="forecast"></a>

Now that we have retrieved the best pipeline/model, it can be used to make predictions on test data.

### Retrieving forecasts from the model

To produce predictions on the test set, we need to know the feature values at all dates in the test set. This requirement is somewhat reasonable for the OJ sales data since the features mainly consist of price, which is usually set in advance, and customer demographics which are approximately constant for each store over the 20 week forecast horizon in the testing data.

Try downloading the model and running forecasts locally.

In [ ]:
import joblib
best_run.download_file('outputs/model.pkl')
model = joblib.load('model.pkl')
model

In [ ]:
# forecast returns the predictions for all specified quantiles
# use 0.025 and 0.975 for the 95% confidence interval
model.quantiles = [0.025, 0.5, 0.975]
y_pred_quantiles = model.forecast_quantiles(test.loc[:, test.columns != target_column_name])
y_pred_quantiles.head()

# Responsible AI Dashboard<a id="analyze"></a>

In [ ]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIForecastingInsights, FeatureMetadata

feature_metadata = FeatureMetadata(
    time_series_id_column_names=time_series_id_column_names, 
    categorical_features=time_series_id_column_names,
    time_column_name=time_column_name)
insights = RAIForecastingInsights(
    model=model,
    train=train,
    test=test,
    target_column=target_column_name,
    feature_metadata=feature_metadata)

ResponsibleAIDashboard(insights)